In [ ]:
%load_ext autoreload
%autoreload 2
import math
from pathlib import Path

import h5py
import numpy as np
from scipy import constants, signal, stats, optimize
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cycler import cycler

import analysis
from basic.paths import (
    RESULTS_FOLDER,
    PARTICLE_VARIATION_FOLDER,
    DENSITY_VARIATION_FOLDER,
    V_FLOW_VARIATION_FOLDER,
    FOLDER_1D,
    FOLDER_2D,
    MPLSTYLE_FILE
)
from basic import (
    physics,
    Species,
    SpeciesInfo,
    RunInfo,
    Distribution
)

from plots import (
    settings,
    plots_1D,
    plots_2D,
    general,
)

info = RunInfo(
    electron=SpeciesInfo(
        number_density=12.0e6,
        temperature=100.0,
        charge=-1,
        mass=1.0,
        bulk_velocity=0.0
    ),
    proton=SpeciesInfo(
        number_density=10.0e6,
        temperature=3.0,
        charge=+1,
        mass=1836.152674,
        bulk_velocity=0.0
    ),
    alpha=SpeciesInfo(
        number_density=1.0e6,
        temperature=12.0,
        charge=+2,
        mass=7294.29953,
        bulk_velocity=1.0e5
    )
)

save=False
plt.style.use(MPLSTYLE_FILE)
matplotlib.rcParams['figure.dpi'] = 100

# Simulation 2D

## Alpha flow-speed variation (B=0)

In [ ]:
plots_2D.maxEnergyVsAlphaFlowSpeed(info, normalize_energy=False)
plt.show()
plots_2D.maxEnergyVsAlphaFlowSpeed(info, normalize_energy=True)
plt.show()

In [ ]:
plots_2D.waveAngleVsAlphaFlowSpeed(info, "x")
plots_2D.simulatedAlphaFlowSpeed(info, "x")
plt.show()

TODO: |k| vs u-alpha

In [ ]:
plots_2D.omegaVsAlphaFlowSpeed(info, "x")
plots_2D.omegaVsAlphaFlowSpeed(info, "y")
plt.show()

In [ ]:
plots_2D.psdOmegaForAlphaFlowSpeed(info, "x")
plots_2D.psdOmegaForAlphaFlowSpeed(info, "y")
plt.show()

In [ ]:
species = Species.ELECTRON
plt.style.use(MPLSTYLE_FILE)
plt.figure()
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    v = int(filename.stem[-3:])
    if v not in [100, 140]:
        continue
    with h5py.File(filename) as f:
        time = f["Header/time"][:] * info.omega_pp
        temp = np.mean(f[f"Derived/Temperature/{species.value}"], axis=(1,2))
    plt.plot(
        time, physics.kelvinToElectronVolt(temp),
        label=f"$u_\\alpha^{{t=0}}$ = {v}"
    )

for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_By").glob("*.h5")):
    v = int(filename.stem[-3:])

    with h5py.File(filename) as f:
        time = f["Header/time"][:] * info.omega_pp
        temp = np.mean(f[f"Derived/Temperature/{species.value}"], axis=(1,2))
    plt.plot(
        time, physics.kelvinToElectronVolt(temp),
        label=f"($B_y>0$) $u_\\alpha^{{t=0}}$ = {v}"
    )

for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx_By").glob("*.h5")):
    v = int(filename.stem[-3:])

    with h5py.File(filename) as f:
        time = f["Header/time"][:] * info.omega_pp
        temp = np.mean(f[f"Derived/Temperature/{species.value}"], axis=(1,2))
    plt.plot(
        time, physics.kelvinToElectronVolt(temp),
        label=f"($B_x=B_y$) $u_\\alpha^{{t=0}}$ = {v}"
    )
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx").glob("*.h5")):
    v = int(filename.stem[-3:])

    with h5py.File(filename) as f:
        time = f["Header/time"][:] * info.omega_pp
        temp = np.mean(f[f"Derived/Temperature/{species.value}"], axis=(1,2))
    plt.plot(
        time, physics.kelvinToElectronVolt(temp),
        label=f"($B_x>0$) $u_\\alpha^{{t=0}}$ = {v}"
    )
plt.legend(title="Flow velocity (km$\\,/\\,$s)")
plt.xlabel("Time $t\\,\\omega_{pp}$ (1)")
plt.ylabel(f"Temperature $T_{species.symbol()}$ (eV)")
plt.xlim(0, 150.0)

In [ ]:
for species in Species:
    plots_2D.temperature3DOverTimeForAlphaFlowSpeed(
        info, species
    )

In [ ]:
for species in Species:
    plots_2D.temperatureDifferences3DVsAlphaFlowSpeed(info, species)
plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.electricField2DSnapshot(filename, info, time=50.0)
    plt.show()

In [ ]:
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_By").glob("*.h5")):
    plots_2D.electricField2DSnapshot(filename, info, time=50.0)
    plt.show()

In [ ]:
files = sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx_By").glob("*.h5"))
arr_theta = np.empty(len(files))
arr_theta_err = np.empty(len(files))
arr_theta_E = np.empty(len(files))
arr_theta_E_err = np.empty(len(files))
flow_velocity = np.empty(len(files))
for file_idx, filename in enumerate(files):
    flow_velocity[file_idx] = int(filename.stem[-3:])
    with h5py.File(filename) as f:
        x = f["Grid/x_px/Alphas/X"][0] / info.lambda_D
        y = f["Grid/y_px/Alphas/Y"][0] / info.lambda_D
        E_x = f['Electric Field/ex'][:]
        E_y = f['Electric Field/ey'][:]
        time = f["Header/time"][:] * info.omega_pp

        print(f.keys())
        break
    res = analysis.fitGrowthRate(time, np.mean(E_x ** 2 + E_y ** 2, axis=(1,2)))
    assert res is not None, "What?"
    linear_idx = res[1]
    E_field = E_x[slice(*linear_idx)]
    k, k_err = analysis.waveVector2D(x, y, E_field)
    print(k, k_err)
    print(f"v = {flow_velocity[file_idx]} :: |k| = {np.linalg.norm(k):.4f}")
    arr_theta[file_idx], arr_theta_err[file_idx] = analysis.waveAngle2DFromWaveVector(k, k_err)

print(arr_theta * 180 / np.pi)

In [ ]:
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx_By").glob("*.h5")):
    plots_2D.energyEFieldOverTime(filename, info)
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.strengthBFieldOverTime(filename, info)
    plots_2D.psdBField(filename, info)
    plots_2D.energyBField(filename, info)
    plt.show()

In [ ]:
for species in [Species.ELECTRON]:
    plots_2D.psdFlowVelocity(info, species, "x", "x")
    plt.show()

In [ ]:
for species in Species:
    plots_2D.flowVelocityVsTime(info, species, "x", "x")
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.energiesOverTime(filename, info)
    plt.show()

In [ ]:
# Kinetic energies in the system
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plt.figure()
    K_species = []
    for species in [Species.ALPHA]:
        u_alpha = int(filename.stem[-3:])
        with h5py.File(filename) as f:
            time = f["Header/time"][:] * info.omega_pp
            x_grid = f[f"Grid/grid/X"][:]
            y_grid = f[f"Grid/grid/Y"][:]
            px_grid = f[f"Grid/px_py/{species.value}/Px"][:]
            py_grid = f[f"Grid/px_py/{species.value}/Py"][:]
            px_py = f[f'dist_fn/px_py/{species.value}'][:]
        # TODO: Wrong
        u_x, u_y = analysis.flowVelocity2D(x_grid, y_grid, px_grid, py_grid, px_py, info[species])
        print(np.sum(u_x, axis=1))
        K = 0.5 * info[species].si_mass * np.mean(u_x ** 2 + u_y ** 2, axis=1) / constants.electron_volt
        K_species.append(info[species].number_density * K)
    for K_s, species in zip(K_species, Species):
        plt.plot(time, K_s * 1e-6, label=f"$W_{{{species.symbol()}}}$")
    # plt.plot(time[1:], W_total[1:] * 1e-6, label="$W_\\text{total}$")
    plt.yscale("log")
    plt.xlabel("Time $t\\,\\omega_{pp}$ (1)")
    plt.ylabel("Kinetic energy density K (MeV$\\,/\\,$m$^3$)")
    plt.xlim(0.0, 150.0)
    plt.legend(ncols=3, labelspacing=.2, columnspacing=0.5)
    plt.show()

In [ ]:
species = Species.PROTON
files = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
labels = [int(p.stem[-3:]) for p in files]
general.momentumDistributionComparison(
    info, species, Distribution.Y_PX, legend_ncols=2,
    files=files, labels=labels, times=150.0, legend_title=f"Flow velocity $u_\\alpha$ (km/s)")
plt.show()

In [ ]:
# TODO: Test with (FOLDER_2D / "v_alpha_bulk_variation_Bx_By") (varying momentum limits)
times = [0.0, 40.0, 50, 60.0, 150.0]
species = Species.PROTON
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    general.momentumDistributionComparison(info, species, Distribution.Y_PX, filename, times)
plt.show()

In [ ]:
species = Species.ALPHA
dist = Distribution.X_PX
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    general.spaceMomentumDistributon(info, species, dist, filename, time=50.0)
    plt.show()

In [ ]:
species = Species.PROTON
time = 150.0
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.pxPyDistribution(info, species, filename, time)
    plt.show()

In [ ]:
filename = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))[2]
plots_2D.videoEFieldOverTime(info, filename, "x", save=True)

In [ ]:
filenames = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
species = Species.PROTON
normalized_velocity = True
time_steps = range(0, 1500, 20)
labels = [int(p.stem[-3:]) for p in filenames]
legend_title=f"Flow velocity $u_\\alpha$ (km/s)"
general.videoMomentumDistribution(
    info, Distribution.X_PX, species, filenames,
    save=True, labels=labels, legend_title=legend_title,
    legend_ncols=2)

In [ ]:
filename = RESULTS_FOLDER / "epoch_2D/v_alpha_bulk_variation/v_alpha_bulk_106.h5"
plots_2D.videoPxPyDistribution(info, species=Species.PROTON, filename=filename)

In [ ]:
filename = RESULTS_FOLDER / "epoch_2D/v_alpha_bulk_variation/v_alpha_bulk_106.h5"
v_x, v_y, f_v = general._loadPxPyDistribution(info, Species.PROTON, filename, 100, True)
f_v[f_v<=0] = np.min(f_v[f_v>0])
from scipy import ndimage
plt.contourf(np.log(ndimage.gaussian_filter(f_v, sigma=10)))
plt.gca().set_aspect("equal")

# Simulation 1D

## Density variation

In [ ]:
for filename in sorted(DENSITY_VARIATION_FOLDER.glob("density_*.h5")):
    print(filename)
    ratio = 10 ** float(filename.stem.split("_")[-1])
    n_electron = 12e6
    n_proton = n_electron / (1 + 2 * ratio)
    n_alpha = ratio * n_proton
    info = RunInfo(
        electron=SpeciesInfo(
            number_density=12.0e6,
            temperature=100.0,
            charge=-1,
            mass=1.0,
            bulk_velocity=0.0
        ),
        proton=SpeciesInfo(
            number_density=n_proton,
            temperature=3.0,
            charge=+1,
            mass=1836.152674,
            bulk_velocity=0.0
        ),
        alpha=SpeciesInfo(
            number_density=n_alpha,
            temperature=12.0,
            charge=+2,
            mass=7294.29953,
            bulk_velocity=1.0e5
        )
    )
    plots_1D.energyEFieldOverTime(
        filename=filename, info=info
    )
    plt.show()

## Run n=8192

In [ ]:
plots_1D.electricFieldOverSpaceAndTime(
    filename=FOLDER_1D / "proton-alpha-instability-1D.h5",
    info=info, save=False
)
plt.show()

In [ ]:
plots_1D.averageTemperature3DOverTime(
    filename=FOLDER_1D / "proton-alpha-instability-1D.h5",
    info=info, save=save
)
plt.show()

In [ ]:
for s in Species:
    plots_1D.velocityDistributionOverTime(
        filename=PARTICLE_VARIATION_FOLDER / "particles_4192/rep_0.h5", #FOLDER_1D / "proton-alpha-instability-1D.h5",
        info=info, species=s, save=save
    )
plt.show()

In [ ]:
plots_1D.energyEFieldOverTime(
    filename=FOLDER_1D / "proton-alpha-instability-1D.h5",
    info=info, save=save
)
plt.show()

## Particle variation

In [ ]:
variation_folder = PARTICLE_VARIATION_FOLDER
for particle_folder in sorted(variation_folder.glob("particles_*")):
    plots_1D.multiElectricFieldEnergyOverTime(
        particle_folder, info, save=save,
        identifer=f"particles_per_cell_{int(particle_folder.stem[-4:])}"
    )

In [ ]:
plots_1D.particleVariationEnergyVsTime(info=info, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationGrowthRate(info, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperature3D(species, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXDiff(info, species, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXVsTime(info, species, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationWavenumber(info=info, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationFrequency(info=info, save=False)
plt.show()